# 8장. 핵심 키워드 추출 (Keyword Extraction)

# 8-0 데이터 준비

## Mecab 설치 (필요시)

In [5]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy    
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.0MB/s 
     |████████████████████████████████| 450kB 37.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.

# 8-1 TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용


In [2]:
import requests 
from bs4 import BeautifulSoup
import re

def get_news_by_url(url):
  headers={"user-agent":"Mozilla/5.0"}
  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.content, "html.parser")
  content = soup.select_one("#articleBodyContents").get_text().replace("\n", "")
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
  
  start_pos = re.search(r"\w+@\w+\.\w+(.\w+)?", content).start()
  content = content[:start_pos-1]
  return content

docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

5

### 1) 전처리

In [3]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []


for doc in docs:
  token_list = []
  for token in mecab.pos(doc):
    if token[1] in ['NNG', 'VV']:
      token_list.append(token[0])
  preprocessed_docs.append(" ".join(token_list))


preprocessed_docs[0][:100]

'과기 정통부 장관 참석 기념행사 투입 여종 데이터 구축 민간 외부 연계 체계 개방 강화 기자 국가 차원 빅 데이터 활용 시대 산업 창출 기존 산업 변화 이르 혁신 장 센터 문 분야'

### 2) TF-IDF 계산

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.85, max_features=10000)
word_count = count_vect.fit_transform(preprocessed_docs)
print((count_vect.get_feature_names()[:10]))


['가공', '가능', '가입자', '가족', '가중치', '가치', '각종', '감소', '감염', '강국']


In [7]:
print(type(count_vect.fit(preprocessed_docs)))
print(type(count_vect.fit_transform(preprocessed_docs)))

<class 'sklearn.feature_extraction.text.CountVectorizer'>
<class 'scipy.sparse.csr.csr_matrix'>


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count)


TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

### 3) 핵심키워드 추출

In [20]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [ ]:
tfidf_vect.tocoo().col

array([391, 387, 385, 382, 380, 379, 378, 377, 375, 374, 373, 372, 371,
       370, 369, 365, 362, 361, 360, 359, 358, 355, 354, 352, 349, 347,
       346, 345, 338, 337, 336, 335, 333, 332, 330, 328, 327, 326, 325,
       323, 321, 320, 319, 317, 316, 314, 308, 307, 306, 301, 299, 294,
       289, 288, 286, 285, 284, 283, 282, 281, 280, 276, 274, 270, 268,
       265, 263, 259, 256, 251, 247, 246, 245, 241, 240, 239, 237, 234,
       233, 231, 229, 228, 227, 226, 223, 222, 221, 219, 216, 214, 213,
       212, 211, 210, 209, 204, 203, 198, 195, 194, 192, 189, 185, 184,
       182, 181, 180, 179, 178, 177, 175, 172, 171, 166, 163, 162, 161,
       160, 152, 151, 150, 147, 146, 144, 142, 139, 135, 134, 132, 129,
       128, 126, 122, 121, 118, 115, 113, 110, 109, 108, 107, 105, 103,
       102, 101, 100,  98,  95,  91,  88,  87,  86,  83,  81,  80,  77,
        76,  75,  73,  71,  68,  67,  64,  62,  61,  59,  58,  55,  54,
        52,  51,  49,  48,  47,  46,  45,  43,  41,  38,  37,  3

In [ ]:
tfidf_vect.tocoo().data

array([0.03703823, 0.18780025, 0.07407646, 0.07407646, 0.03703823,
       0.03703823, 0.03703823, 0.02988221, 0.03703823, 0.14941106,
       0.03703823, 0.03703823, 0.02988221, 0.03703823, 0.07407646,
       0.03703823, 0.05976443, 0.03703823, 0.25926763, 0.03703823,
       0.02988221, 0.03703823, 0.02988221, 0.02988221, 0.03703823,
       0.1111147 , 0.03703823, 0.1111147 , 0.03703823, 0.02988221,
       0.03703823, 0.07407646, 0.03703823, 0.04173339, 0.02988221,
       0.07407646, 0.03703823, 0.03703823, 0.1111147 , 0.03703823,
       0.1111147 , 0.03703823, 0.03703823, 0.02988221, 0.03703823,
       0.03703823, 0.02988221, 0.02988221, 0.03703823, 0.03703823,
       0.1111147 , 0.07407646, 0.02988221, 0.03703823, 0.12402468,
       0.07407646, 0.18519116, 0.02988221, 0.04173339, 0.02988221,
       0.08964664, 0.07407646, 0.03703823, 0.03703823, 0.03703823,
       0.02988221, 0.1111147 , 0.03703823, 0.03703823, 0.02988221,
       0.03703823, 0.03703823, 0.03703823, 0.03703823, 0.02480

In [21]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vect.get_feature_names()
tfidf_vect = tfidf_transformer.transform(count_vect.transform([doc]))
sorted_keywords = sort_keywords(tfidf_vect.tocoo())


# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)
print(keywords)
print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)

[('플랫', 0.25926762688545546), ('센터', 0.22953363480835778), ('계획', 0.22222939447324755), ('활용', 0.18780024666138362), ('정통부', 0.18519116206103964)]

===== 원문 =====
과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자

=== 핵심키워드 ===
('플랫', 0.25926762688545546)
('센터', 0.22953363480835778)
('계획', 0.22222939447324755)
('활용', 0.18780024666138362)
('정통부', 0.18519116206103964)


In [16]:
tfidf_vect.tocoo().toarray()

array([[0.        , 0.02086669, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.02988221, 0.        , 0.03703823,
        0.        , 0.03703823, 0.03703823, 0.        , 0.08964664,
        0.03703823, 0.        , 0.        , 0.02988221, 0.        ,
        0.        , 0.03703823, 0.        , 0.        , 0.02480494,
        0.07407646, 0.11952885, 0.        , 0.03703823, 0.22222939,
        0.03703823, 0.04173339, 0.03703823, 0.07407646, 0.03703823,
        0.02988221, 0.03703823, 0.18519116, 0.05976443, 0.        ,
        0.        , 0.03703823, 0.        , 0.07407646, 0.        ,
        0.07407646, 0.03703823, 0.17929328, 0.03703823, 0.02988221,
        0.        , 0.03703823, 0.02988221, 0.        , 0.03703823,
        0.08964664, 0.        , 0.        , 0.02480494, 0.03703823,
        0.        , 0.05976443, 0.04960987, 0.        , 0.14941106,
        0.        , 0.        , 0.03703823, 0.05976443, 0.        ,
        0.        , 0.03703823, 0.        , 0.03

In [ ]:
tf_idf_vector.tocoo().data

array([0.03608783, 0.03608783, 0.05823087, 0.07217566, 0.03608783,
       0.03608783, 0.03608783, 0.02911543, 0.03608783, 0.14557716,
       0.03608783, 0.03608783, 0.02911543, 0.02911543, 0.07217566,
       0.11646173, 0.03608783, 0.03608783, 0.04833688, 0.03608783,
       0.03608783, 0.2526148 , 0.03608783, 0.02911543, 0.03608783,
       0.02911543, 0.02911543, 0.03608783, 0.10826349, 0.1746926 ,
       0.07217566, 0.03608783, 0.10826349, 0.03608783, 0.07217566,
       0.03608783, 0.02911543, 0.02911543, 0.07217566, 0.03608783,
       0.04066251, 0.02911543, 0.07217566, 0.03608783, 0.03608783,
       0.10826349, 0.03608783, 0.03608783, 0.10826349, 0.03608783,
       0.03608783, 0.02911543, 0.03608783, 0.03608783, 0.02911543,
       0.02911543, 0.03608783, 0.03608783, 0.10826349, 0.02911543,
       0.07217566, 0.03608783, 0.02911543, 0.03608783, 0.1208422 ,
       0.07217566, 0.18043914, 0.02911543, 0.04066251, 0.02911543,
       0.09667376, 0.07217566, 0.03608783, 0.02033125, 0.02911


---


## 실습 2. gensim 활용


### 1) 전처리

In [22]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [23]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(valuue) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

### 3) 핵심키워드 추출

In [24]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [25]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


=== 핵심키워드 ===
('플랫', 0.2495222182663338)
('폼', 0.2495222182663338)
('계획', 0.21387618708542896)
('정통부', 0.17823015590452412)
('위한', 0.17823015590452412)


In [ ]:
tfidf[doc]

[(0, 0.021155348483460852),
 (1, 0.011793957648078673),
 (3, 0.023587915296157346),
 (4, 0.021155348483460852),
 (5, 0.037158751819300714),
 (6, 0.037158751819300714),
 (7, 0.037158751819300714),
 (8, 0.036063616033346915),
 (9, 0.06346604545038255),
 (10, 0.037158751819300714),
 (11, 0.042310696966921704),
 (12, 0.021155348483460852),
 (13, 0.037158751819300714),
 (14, 0.021155348483460852),
 (15, 0.037158751819300714),
 (16, 0.021155348483460852),
 (17, 0.005151945147620987),
 (18, 0.07431750363860143),
 (19, 0.10577674241730427),
 (20, 0.037158751819300714),
 (21, 0.2229525109158043),
 (22, 0.037158751819300714),
 (23, 0.010303890295241975),
 (24, 0.037158751819300714),
 (25, 0.07431750363860143),
 (26, 0.037158751819300714),
 (27, 0.011793957648078673),
 (28, 0.037158751819300714),
 (29, 0.18579375909650356),
 (30, 0.042310696966921704),
 (31, 0.037158751819300714),
 (32, 0.042310696966921704),
 (33, 0.07431750363860143),
 (34, 0.011793957648078673),
 (35, 0.037158751819300714),
 (



---



# 8-2 Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 실습 1. 행렬 활용 


In [33]:
tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

# [vocab2idx[token] for token in vocab] 

vocab2idx = {vocab[i]:i for i in range(0, len(vocab))} #vocab을 인덱스로 변환
idx2vocab = {i:vocab[i] for i in range(0, len(vocab))} #인덱스를 vocab으로 변환
vocab2idx

{'딸기': 3, '바나나': 0, '사과': 1, '파인애플': 2}

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [34]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len),dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2
covered_cooccurence = []

tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']

for window_start in range(0, (len(tokens) - window_size + 1)):
    window = tokens[window_start : window_start + window_size]
    
    for i in range(window_size):
        for j in range(i + 1, window_size):
            if(window[i] in vocab and window[j] in vocab):
                index_i = i + window_start
                index_j = j + window_start
                
                if (index_i, index_j) not in covered_cooccurence:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_cooccurence.append((index_i, index_j))


for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    print(f"{i} : {row_sum}")
    weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0

print(weighted_edge)

0 : 2.0
1 : 2.0
2 : 1.0
3 : 3.0
[[0.         0.5        0.         0.5       ]
 [0.5        0.         0.         0.5       ]
 [0.         0.         0.         1.        ]
 [0.33333334 0.33333334 0.33333334 0.        ]]


### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [35]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(MAX_ITERATIONS):
    prev_score = np.copy(score)

    for i in range(vocab_len):
        summation = 0
        for j in range(vocab_len):
            summation += weighted_edge[j][i] * prev_score[j]

        score[i] = (1-d) + d*summation

    if np.sum(np.fabs(prev_score - score)) <= threshold:
        break

print(score)


[0.98364776 0.98365426 0.5656039  1.4668667 ]


### 5) 핵심 단어 추출

In [36]:
sorted_index = np.flip(np.argsort(score),0)

n = 4

print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
딸기 : 1.4668667
사과 : 0.98365426
바나나 : 0.98364776
파인애플 : 0.5656039


In [ ]:
sorted_index[0]

3

In [3]:
import requests 
from bs4 import BeautifulSoup
import re

def get_news_by_url(url):
  headers={"user-agent":"Mozilla/5.0"}
  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.content, "html.parser")
  content = soup.select_one("#articleBodyContents").get_text().replace("\n", "")
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
  
  start_pos = re.search(r"\w+@\w+\.\w+(.\w+)?", content).start()
  content = content[:start_pos-1]
  return content

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc = re.sub("[^가-힣 \d]", " ", doc)
doc

'과기정통부  22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입  5100여종 데이터 구축민간 클라우드 통한 외부연계체계도   개방성 강화  이데일리 이재운 기자  국가 차원의 빅데이터 활용 시대가 열린다  새로운 산업 창출과 기존 산업의 변화에 이르는  혁신성장 을 위한 센터가 문을 연다  10개 분야에 걸쳐  데이터 경제 의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다 22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한  빅데이터 플랫폼 및 센터  출범식 행사를 개최했다  유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다  10개 분야 100개 센터  3년간 1516억원 투입이미지  픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다  문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호 보안 에 대한 중요성을 강조하기도 했다 이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산 구축하고  플랫폼은 이를 수집 분석 유통하는 역할을 담당한다  과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며  올해 우선 640억원 규모의 사업을 추진하고 있다 대상 분야는  금융   카드   환경 한국수자원공사   문화 한국문화정보원   교통 한국교통연구원   헬스케어 국립암센터   유통 소비 매일방송   통신      중소기업 더존비즈온   지역경제 경기도청   산림 한국임업진흥원  등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고  다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원  운영할 계획이다  이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다 주요 활용 전략 사례를 보면 빅데이터 활용을 

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [8]:
from konlpy.tag import Mecab
mecab = Mecab()

tokens = [ token for token in mecab.pos(doc) ]
nodes = [t[0] for t in tokens]
vocab = [t[0] for t in tokens if t[1] in ['NNG', 'NNP'] and len(t[0]) > 1]

print(nodes[:10])
print(vocab[:10])



['과기', '정통부', '22', '일', '유영민', '장관', '등', '참석', '해', '기념행사']
['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '데이터', '구축']


### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [9]:
 vocab = list(set(vocab))

 vocab2idx = {vocab[i]:i for i in range(len(vocab))}
 idx2vocab = {i:vocab[i] for i in range(len(vocab))}

In [47]:

import numpy as np
import math

vocab_len = len(vocab2idx)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len),dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 3
covered_coocurrences = []

for window_start in range(len(nodes) - window_size + 1):
  window = nodes[window_start:window_start+window_size]
  for i in range(window_size):
    for j in range(i+1, window_size):
      if window[i] in vocab and window[j] in vocab:
        index_i = window_start + i
        index_j = window_start + j

        if (index_i, index_j) not in covered_coocurrences:
          weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
          weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
          covered_coocurrences.append((index_i, index_j))

for i in range(vocab_len):
  row_sum = weighted_edge[i].sum()
  weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0

MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(MAX_ITERATIONS):
  prev_score = np.copy(score)

  for i in range(vocab_len):
    summation = 0
    for j in range(vocab_len):
      if weighted_edge[j][i] != 0:
        summation += weighted_edge[j][i] * prev_score[j]

    score[i] = (1 - d) * d*summation

  if np.sum(np.fabs(prev_score -  score)) <= threshold:
    break


sorted_index = np.flip(np.argsort(score), 0)

n = 5


print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
데이터 : 7.405189e-07
센터 : 2.49831e-07
한국 : 2.0378042e-07
활용 : 1.9834519e-07
대한 : 1.9389464e-07


---

## 실습 2. 그래프 활용 

### 2) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 없음을 의미
*   모든 노드는 1로 초기화

In [15]:
import numpy as np
import math
import networkx as nx

# 윈도내 동시 등장한 토큰으로 그래프를 생성
def connect(vocab, nodes) :            
    window_size = 3

    edges = []
    for window_start in range(len(nodes) - window_size + 1):
      window = nodes[window_start: window_start + window_size]
      for i in range(window_size):
        for j in range(i + 1, window_size):
          if window[i] in vocab and window[j] in vocab:
            edges.append((window[i], window[j]))
      
    return edges


graph=nx.diamond_graph()
graph.clear() 
graph.add_nodes_from(vocab) #node 등록
graph.add_edges_from(connect(vocab, nodes)) #edge 연결    

### 3) 스코어 계산 및 핵심키워드 추출

In [19]:
scores = nx.pagerank(graph) #pagerank 계산

rank = sorted(scores.items(), key=lambda x: x[1], reverse=True) #score 역순 정렬
print("\n=== 핵심키워드 ===")
rank[:10]


=== 핵심키워드 ===


[('데이터', 0.04661070207125959),
 ('센터', 0.01676522225593208),
 ('대한', 0.01241381183474908),
 ('활용', 0.01236131939785426),
 ('한국', 0.012348645054362992),
 ('문화', 0.011604486043084145),
 ('경제', 0.011216010863143161),
 ('제공', 0.010178677656595397),
 ('구축', 0.00991838441569345),
 ('통신', 0.009755398667462634)]

---

## 실습 3. TextRank 핵심 구 추출

### 1) 불용어를 기준으로 구 추출

In [ ]:
phrases = []
phrase = " "


print(phrases)

['과기 정통부', '일 유영민 장관 등 참석', '기념행사', '년', '억 원 투입', '여종 데이터 구축 민간 클라우드', '외부 연계 체계', '개방', '강화', '데일리 이재운 기자', '국가 차원', '빅 데이터 활용 시대', '산업 창출', '기존 산업', '변화', '혁신', '장', '센터', '문', '개 분야', '데이터 경제', '발전', '정부', '청사진', '현실', '구현', '데', '계획', '일 과학 기술 정보 통신부', '서울 중구 대한 상공 회의소', '데이터 생태', '조성', '혁신 성장', '기반 마련', '빅 데이터 플랫 폼', '센터', '출범식 행사', '개최', '유영민 과기 정통부 장관', '노웅래 국회 과학 기술 정보 방송 통신 위원회 위원장 등', '명', '참가', '개 분야', '개 센터', '년 간', '억 원 투입 이미지', '픽사 베이 빅 데이터', '데이터 활용', '혁신', '장', '문재', '정부', '경제 성장 핵심 요소 중 하나', '문재인 대통령', '데이터 활용', '이', '정보 보호', '보안', '중요', '강조', '맥락 속', '빅 데이터 센터', '공공', '민간', '협업', '활용', '양질', '데이터', '생산', '구축', '플랫 폼', '이', '수집', '분석', '유통', '역할', '담당', '과기 정통부', '분야', '플랫 폼', '개소', '이', '연계', '기관', '센터', '개소', '구축', '데', '년', '억 원', '투입', '계획', '올해', '억 원 규모', '사업', '추진', '대상 분야', '금융', '카드', '환경', '한국 수자원 공사', '문화', '한국 문화 정보원', '교통', '한국 교통 연구원', '헬 스케어', '국립암센터', '유통', '소비', '매일 방송', '통신', '중소기업', '비즈', '지역 경제', '경기도 청', '산림', '한국 임업', '흥원', '등', '차 공모', '개 빅 데이터 

In [ ]:
unique_phrases = []


print(unique_phrases)

['과기 정통부', '일 유영민 장관 등 참석', '기념행사', '년', '억 원 투입', '여종 데이터 구축 민간 클라우드', '외부 연계 체계', '개방', '강화', '데일리 이재운 기자', '국가 차원', '빅 데이터 활용 시대', '산업 창출', '기존 산업', '변화', '혁신', '장', '센터', '문', '개 분야', '데이터 경제', '발전', '정부', '청사진', '현실', '구현', '데', '계획', '일 과학 기술 정보 통신부', '서울 중구 대한 상공 회의소', '데이터 생태', '조성', '혁신 성장', '기반 마련', '빅 데이터 플랫 폼', '출범식 행사', '개최', '유영민 과기 정통부 장관', '노웅래 국회 과학 기술 정보 방송 통신 위원회 위원장 등', '명', '참가', '개 센터', '년 간', '억 원 투입 이미지', '픽사 베이 빅 데이터', '데이터 활용', '문재', '경제 성장 핵심 요소 중 하나', '문재인 대통령', '이', '정보 보호', '보안', '중요', '강조', '맥락 속', '빅 데이터 센터', '공공', '민간', '협업', '활용', '양질', '데이터', '생산', '구축', '플랫 폼', '수집', '분석', '유통', '역할', '담당', '분야', '개소', '연계', '기관', '억 원', '투입', '올해', '억 원 규모', '사업', '추진', '대상 분야', '금융', '카드', '환경', '한국 수자원 공사', '문화', '한국 문화 정보원', '교통', '한국 교통 연구원', '헬 스케어', '국립암센터', '소비', '매일 방송', '통신', '중소기업', '비즈', '지역 경제', '경기도 청', '산림', '한국 임업', '흥원', '등', '차 공모', '개 빅 데이터 센터', '선정', '다음 달', '일', '개', '추가 선정', '지원', '운영', '데이터 생태계', '기업', '경쟁력', '제고', '수행', '주요 활용 전략', '사례', '빅 데이

### 2) 각 구의 Score 계산

앞서 산출한 각 단어별 점수를 합산

In [ ]:
vocabs = dict((r[0][0],r[1]) for r in rank)
phrase_scores = []
keywords = []


Keyword: '과기 정통부', Score: 0.01124933498027082
Keyword: '일 유영민 장관 등 참석', Score: 0.013047761313401546
Keyword: '기념행사', Score: 0.0022034464344388585
Keyword: '년', Score: 0
Keyword: '억 원 투입', Score: 0.003215471038656386
Keyword: '여종 데이터 구축 민간 클라우드', Score: 0.06857882292392038
Keyword: '외부 연계 체계', Score: 0.01653296999059789
Keyword: '개방', Score: 0.007608411089591418
Keyword: '강화', Score: 0.0022598290466079364
Keyword: '데일리 이재운 기자', Score: 0.016337448780957005


### 3) 각 구를 Score로 정렬하여 핵심 구 추출

In [ ]:
sorted_index = np.flip(np.argsort(phrase_scores),0)

keywords_num = 10

print("Keywords:\n")

for i in range(0,keywords_num):
    print(str(keywords[sorted_index[i]])+", ")

Keywords:

여종 데이터 구축 민간 클라우드, 
빅 데이터 플랫 폼, 
빅 데이터 센터, 
개 빅 데이터 센터, 
빅 데이터 활용 시대, 
민간 사이 데이터 파일 형식 등, 
빅 데이터 활용, 
픽사 베이 빅 데이터, 
여종 데이터 구축, 
데이터 경제, 




---



## 실습 4. gensim 활용

In [18]:
from gensim.summarization import keywords
keywords(" ".join(vocab), words=5).split('\n')

['국립암센터', '인공지능 얼라이언스', '데일리 연구원']

In [22]:
!pip install gensim==3.6.0

In [24]:
!pip install krwordrank

In [27]:
from krwordrank.sentence import summarize_with_sentences
keywords, sents = summarize_with_sentences([doc], num_keywords=100, num_keysents=10)
print(keywords)

{'데이터': 5.085766373790507, '빅데이터': 2.0618376646478267, '플랫폼': 1.771900307018206, '활용': 1.7221222100354474, '센터': 1.6670032945170843, '통해': 1.5514369726598676, '계획이다': 1.415258303535809, '위한': 1.3181394849870949, '10': 1.00426960122196, '분야': 0.8893027574391277, '기업': 0.8741516095248898, '제공': 0.8275828650756994, '구축': 0.8268010788632445, '과기정통부': 0.7817620416301649, '혁신': 0.692177824146556}


In [28]:
print(keywords)
print(sents)

{'데이터': 5.085766373790507, '빅데이터': 2.0618376646478267, '플랫폼': 1.771900307018206, '활용': 1.7221222100354474, '센터': 1.6670032945170843, '통해': 1.5514369726598676, '계획이다': 1.415258303535809, '위한': 1.3181394849870949, '10': 1.00426960122196, '분야': 0.8893027574391277, '기업': 0.8741516095248898, '제공': 0.8275828650756994, '구축': 0.8268010788632445, '과기정통부': 0.7817620416301649, '혁신': 0.692177824146556}
['과기정통부  22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입  5100여종 데이터 구축민간 클라우드 통한 외부연계체계도   개방성 강화  이데일리 이재운 기자  국가 차원의 빅데이터 활용 시대가 열린다  새로운 산업 창출과 기존 산업의 변화에 이르는  혁신성장 을 위한 센터가 문을 연다  10개 분야에 걸쳐  데이터 경제 의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다 22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한  빅데이터 플랫폼 및 센터  출범식 행사를 개최했다  유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다  10개 분야 100개 센터  3년간 1516억원 투입이미지  픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다  문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호 보안 에 대한 중요성을 강조하기도 했다 이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산 구축하고  플랫폼은 이를 수집 분석 유통하는 

https://github.com/lovit/KR-WordRank